# Problem 74
## Digit factorial chains

The number $145$ is well known for the property that the sum of the factorial of its digits is equal to $145$:

$$1! + 4! + 5! = 1 + 24 + 120 = 145$$

Perhaps less well known is $169$, in that it produces the longest chains of numbers that link back to $169$; it turns out that there are only three such loops that exist:

$$169 \to 363601 \to 1454 \to 169$$

$$871 \to 45361 \to 871$$

$$872 \to 45362 \to 872$$

It is not difficult to prove that EVERY starting number will eventually get stuck in a loop. For example,

$$69 \to 363600 \to 1454 \to 169 \to 363601 (\to 1454)$$

$$78 \to 45360 \to 871 \to 45361 (\to 871)$$

$$540 \to 145 (\to 145)$$

Starting with $69$ produces a chain of five non-repeating terms, but the longest non-repeating chain with a starting number below one million is sixty terms.

How many chains, with a starting number below one million, contain exactly sixty non-repeating terms?

## Solution

In [1]:
from math import factorial, prod
from itertools import combinations_with_replacement
from collections import Counter

In [2]:
def compute(n: int, m: int):
    def get_next_term(k: int) -> int:
        return sum(factorials[digit] for digit in map(int, str(k)))

    def memoize_chain(chain: [int], k: int) -> None:
        index_k = chain.index(k)
        loop_len = chains[k] if k in chains else len(chain) - index_k
        for i, j in enumerate(chain):
            if i >= index_k:
                chains[j] = loop_len
            else:
                chains[j] = index_k - i + loop_len

    def get_chain_len(k: int) -> int:
        if k in chains:
            return chains[k]
        term = k
        chain = [term]
        term = get_next_term(term)
        while term not in chain:
            chain.append(term)
            if term in chains:
                break
            term = get_next_term(term)
        memoize_chain(chain, term)
        return chains[k]
    factorials = [factorial(i) for i in range(10)]
    chains = dict()
    result = 0
    for combination in combinations_with_replacement('9876543210', n):
        combination = list(combination)
        for _ in range(combination.count('0') + 1):
            if get_chain_len(int(''.join(combination))) == m:
                n_digits = len(combination)
                n_zeros = combination.count('0')
                counter = Counter(combination)
                result += (n_digits - n_zeros) * factorials[n_digits - 1] // prod([int(i) for i in counter.values()])
            if '0' in combination:
                combination.remove('0')
                if len(combination) == 0:
                    break
    return result

In [3]:
compute(6, 60)

402

In [4]:
%timeit -n 100 -r 1 -p 6 compute(6, 60)

32.2714 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)
